# Model based instrumental variables estimation
_Bryan Graham - University of California - Berkeley_  

October 2019

#### Code citation:
<br>
Graham, Bryan S. (2019). "Model based instrumental variables estimation: Python Jupyter Notebook," (Version 1.0) [Computer program]. Available at http://bryangraham.github.io/econometrics/ (Accessed 19 March 2020)

### Overview
This notebook describes an implementation of a so called model based approach to the binary treatment, binary instrument, model described in, for example, Angrist, Imbens and Rubin (1996). Specifically it implements the parametric estimator proposed by Imbens and Rubin (1997). This is a maximum likelihood (MLE) estimator which recovers the potential outcome distribution, under both treatment and control, for the latent subpopulation of _compliers_ (units induced to change their treatment by the instrument). The potential outcome distributions, under control for _never takers_ , and under treatment for _always takers_ , are also recovered. The MLE is computed using the EM-Algorithm. Standard errors are constructed using either (i) a BFGS approximation to the (observed) log-likelihood Hessian matrix or (ii) a parametric bootstrap procedure. The latter method is recommended for empirical work.
<br>
<br>
A textbook description of model-based instrumental variables analysis is provided by Imbens and Rubin (2015, Chapter 25). Relative to the the method-of-moments instrumental variables estimator, a model-based approach involves parametric modelling of potential outcome distributions across the various compliance strata. The trade-off here is greater efficiency at the cost of a potential lack of robustness to these extra distributional assumptions. Also relative to the method-of-moments approach, it is easier to incorporate additional covariates into a model-based analysis. In practice this can result in a much more flexible empirical analysis than is typical when using method-of-moments instrumental variables estimators. The implementation here assumes the potential outcome distributions are conditionally Gaussian given covariates and the latent compliance strata. Other distributional assumptions could be made.
<br>
<br>
To illustrate the model based approach in practice, it is used to estimate the return to a 4-year college degree using the geographic proximity instrument of Card (1995). This analysis is not intended to be a replication analysis, nor a serious analysis of the returns to a college degree, it simply intended to provide a hands on illustration of model-based IV methods.
<br>
<br>
**References**
<br>
<br>
[1] Angrist, Joshua D., Guido W. Imbens and Donald B. Rubin. (1996). "Identification of causal effects using instrumental variables," _Journal of the American Statistical Association_ 91 (434): 444 - 455
<br>
<br>
[2] Card, David. (1995). "Using geographic proximity to estimate the return to schooling," _Aspects of Labor Market Behavior: Essays in Honour of John Vanderkamp_ (L. N. Christofides, E. K. Grant and R. Swidinsky, Eds.).: 201 - 224. Toronto: University of Toronto Press.
<br>
<br>
[3] Imbens, Guido W. and Donald B. Rubin. (1997). "Estimating outcome distributions for compliers in instrumental variable models," _Review of Economic Studies_ 64 (4): 555 - 574.
<br>
<br>

I begin by loading a standard set of scientific computing libraries.

In [1]:
# Load library dependencies
import numpy as np
import numpy.linalg
import numpy.matlib

import scipy as sp
import scipy.optimize
import scipy.stats

import pandas as pd
pd.set_option('precision',4)

import time

Use the basic *ols* and *iv* commands available in my _ipt_ module.

In [2]:
# Append location of ipt module base directory to system path
# NOTE: only required if permanent install not made (see comments above)
import sys
sys.path.append('/Users/bgraham/Dropbox/Sites/software/ipt/')

# import ols function from the ipt module
from ipt import ols, iv

The Card (1995) dataset is available [online](http://davidcard.berkeley.edu/data_sets.html). This next block of code loads the dataset into memory and extracts an estimation sub-sample. This sub-sample is a subset of the one summarized in Column 3 of Table 1 in Card (1995, p. 203). Specially I additionally exclude black and southern respondents respondents as well as respondents with less than a high school degree. This leads to an estimation sample $ 1480 $ respondents.

In [3]:
# Directory where supply-chain dataset source files are located
data =  '/Users/bgraham/Dropbox/Teaching/Berkeley_Courses/Ec240a/Ec240a_Fall2019/ProblemSets/Card_RLE95_Data/'

In [4]:
card = pd.read_fwf(data + 'nls.dat', na_values = '.', header=None)
card = card[[0,2,5,7,8,9,10,11,12, 13,14,16,17,18,19,20,21,22,23,25,28,31,37]]
card.rename(columns={0: "id", 2: "nearc4", 5: "ed76", \
                     7: "age76", 8: "daded", 9: "nodaded", 10: "momed", 11: "nomomed", \
                     12: "weight", 13: "momdad14", 14: "sinmom14", \
                     16: "reg661", 17: "reg662", 18: "reg663", 19: "reg664", \
                     20: "reg665", 21: "reg666", 22: "reg667", 23: "reg668", \
                     25: "south66", 28: "lwage76", 31: "black", 37: "smsa66r"}, inplace=True)
card.dropna(how = 'any', inplace=True)
card = card[(card.ed76 >= 12) & (card.black != 1) & (card.south66 != 1)]

card.describe()

,id,nearc4,ed76,age76,daded,nodaded,momed,nomomed,weight,momdad14,...,reg663,reg664,reg665,reg666,reg667,reg668,south66,lwage76,black,smsa66r
count,1480.0000,1480.0000,1480.0000,1480.0000,1480.0000,1480.0000,1480.0000,1480.0000,1480.0000,1480.0000,...,1480.0000,1480.0000,1480.0,1480.0,1480.0,1480.0000,1480.0,1480.0000,1480.0,1480.0000
mean,483.4189,0.7682,14.2608,28.1993,10.8973,0.1574,11.1894,0.0736,385057.7514,0.8595,...,0.3162,0.1149,0.0,0.0,0.0,0.0534,0.0,6.3930,0.0,0.7358
std,307.4624,0.4221,2.1435,3.1492,2.7801,0.3643,2.4900,0.2613,97957.7163,0.3477,...,0.4652,0.3190,0.0,0.0,0.0,0.2249,0.0,0.4141,0.0,0.4410
min,0.0000,0.0000,12.0000,24.0000,0.0000,0.0000,0.0000,0.0000,77455.0000,0.0000,...,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,4.7622,0.0,0.0000
25%,199.7500,1.0000,12.0000,26.0000,9.9400,0.0000,10.0000,0.0000,358554.0000,1.0000,...,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,6.1356,0.0,0.0000
50%,478.0000,1.0000,14.0000,28.0000,11.0000,0.0000,12.0000,0.0000,370732.0000,1.0000,...,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,6.4370,0.0,1.0000
75%,765.0000,1.0000,16.0000,31.0000,12.0000,0.0000,12.0000,0.0000,418230.0000,1.0000,...,1.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,6.6451,0.0,1.0000
max,999.0000,1.0000,18.0000,34.0000,18.0000,1.0000,18.0000,1.0000,993271.0000,1.0000,...,1.0000,1.0000,0.0,0.0,0.0,1.0000,0.0,7.7849,0.0,1.0000


In [5]:
# Cross sectional sampling weight
sw = card['weight']

# outcome: log wages in 1976
Y = card['lwage76']

# treatment: completed 4-year college degree
D = (card['ed76']>=16)*1

# instrument: lived near 4-year college
Z = card['nearc4']

# pre-treatment control variables
X = card[['age76', 'smsa66r']]
X = X.assign(constant=1)

# combined instrument and controls
ZX = X.copy(deep=True)
ZX.insert(loc=0, column="nearc4", value=Z)

# combined treatment and controls
DX = X.copy(deep=True)
DX.insert(loc=0, column="college", value=D)
DX.describe()

,college,age76,smsa66r,constant
count,1480.0000,1480.0000,1480.0000,1480.0
mean,0.3662,28.1993,0.7358,1.0
std,0.4819,3.1492,0.4410,0.0
min,0.0000,24.0000,0.0000,1.0
25%,0.0000,26.0000,0.0000,1.0
50%,0.0000,28.0000,1.0000,1.0
75%,1.0000,31.0000,1.0000,1.0
max,1.0000,34.0000,1.0000,1.0


I start by computing the OLS fit of log wages onto the college dummy and pre-treatment controls. For this calculation, as well as all that follow, the NLS 1976 cross-sectional sampling weights are used. Compare these results with Table 2 of Card (1995, p. 206). To keep the analysis simple I only include the respondent's age and whether they resided in a standard metropolitan statistical area (SMSA) in 1966 as controls (college proximity covaries with this latter control). The estimated earnings premium associated with a college degree is just 9 percent.

In [7]:
_ = ols(Y, DX, c_id=None, s_wgt=sw, nocons=True, silent=False)


-----------------------------------------------------------------------
-                     OLS ESTIMATION RESULTS                          -
-----------------------------------------------------------------------
Dependent variable:        lwage76
Number of observations, n: 1480



Independent variable       Coef.    ( Std. Err.)     (0.95 Confid. Interval )
-------------------------------------------------------------------------------------------
college                    0.090294 (  0.021156)     (  0.048829 ,  0.131760)
age76                      0.048128 (  0.003313)     (  0.041633 ,  0.054622)
smsa66r                    0.106975 (  0.022544)     (  0.062790 ,  0.151160)
constant                   4.928739 (  0.094401)     (  4.743716 ,  5.113762)

-------------------------------------------------------------------------------------------
NOTE: Heteroscedastic-robust standard errors reported
NOTE: (Sampling) Weighted estimates computed.
      Weight-variable   = weight


Next I compute the first stage fit of college onto the proximity indicator and controls. Compare with Table 3 of Card (1995, p. 209). The results suggest that, on average, high school graduates residing near a four-year college were about 13 percentage points more likely to get a college degree. This effect is not precisely determined (the 95 percent confidence interval is from 0.06 to 0.19).

In [8]:
_ = ols(D, ZX, c_id=None, s_wgt=sw, nocons=True, silent=False)


-----------------------------------------------------------------------
-                     OLS ESTIMATION RESULTS                          -
-----------------------------------------------------------------------
Dependent variable:        ed76
Number of observations, n: 1480



Independent variable       Coef.    ( Std. Err.)     (0.95 Confid. Interval )
-------------------------------------------------------------------------------------------
nearc4                     0.126321 (  0.032697)     (  0.062236 ,  0.190406)
age76                      0.008646 (  0.004096)     (  0.000618 ,  0.016675)
smsa66r                   -0.043407 (  0.032453)     ( -0.107014 ,  0.020201)
constant                   0.063506 (  0.117325)     ( -0.166446 ,  0.293459)

-------------------------------------------------------------------------------------------
NOTE: Heteroscedastic-robust standard errors reported
NOTE: (Sampling) Weighted estimates computed.
      Weight-variable   = weight


Next I compute the standard "method-of-moments" or linear IV estimate of the return to a college degree. Compare with Table 4 of Card (1995. p. 211). The estimated return is very imprecisely determined.

In [9]:
_ = iv(Y, DX, ZX, c_id=None, s_wgt=sw, nocons=True, silent=False)


-----------------------------------------------------------------------
-                    LINEAR IV/2SLS ESTIMATION RESULTS                -
-----------------------------------------------------------------------
Dependent variable:        lwage76
Number of observations, n: 1480


Independent variable       Coef.    ( Std. Err.)     (0.95 Confid. Interval )
-------------------------------------------------------------------------------------------
college                    0.116309 (  0.205476)     ( -0.286416 ,  0.519033)
age76                      0.047883 (  0.003898)     (  0.040243 ,  0.055522)
smsa66r                    0.106783 (  0.022550)     (  0.062586 ,  0.150980)
constant                   4.926100 (  0.095855)     (  4.738228 ,  5.113972)

-------------------------------------------------------------------------------------------
NOTE: Heteroscedastic-robust standard errors reported
NOTE: (Sampling) Weighted estimates computed.
      Weight-variable   = weight
------

In [10]:
def LATE_mle(D, Y, X, Z, cLATE=False, nocons=True, s_wgt=None, silent=False, \
             theta_sv=None, gtol=1e-2, xtol=1e-4, maxiter=1000, skipHess=False):
    
    """
    AUTHOR: Bryan S. Graham, UC - Berkeley, bgraham@econ.berkeley.edu
    DATE: 8 Oct 2019    
    
    OVERVIEW
    --------
    
    This function implements a model-based approach to Local Average
    Treatement Effect estimation along the lines of Imbens and Rubin (1997, ReStud).
    MLE mixture model methods are used; estimation proceeds using the EM-Algorithm. 
    An introduction to model-based methods of instrumental variable analysis is
    provided by Imbens and Rubin (2015, Chapter 25). Users should start routine with
    multiple sets of starting values to ensure that a global maximum of the mixture
    log-likelihood has been found.
        
    INPUTS:
    -------
    D        : n X 1 pandas.Series of binary treatment
    Y        : n X 1 pandas.Series of outcome variable
    X        : n X K pandas.DataFrame of (functions of) pre-treatment control variables 
    Z        : n X 1 pandas.Series of binary instrument
    cLATE    : If True impose E[Y(1) - Y(0)|X,A=c] equals a constant (i.e., no X interactions)  
    nocons   : if True do not add a constant vector to X matrix of pre-treatment control variables
               (if a constant is included program assumes that is in the last column of X) 
    s_wgt    : n X 1 pandas.Series of sampling weights  
    silent   : if set equal to True, then suppress all estimation output 
    theta_sv : starting values for parameter vector (try alternatives in addition to default)
    gtol     : optimization stopping criterion: change in logL
    xtol     : optimization stopping criterion: norm of change in parameter values
    maxiter  : optimization stopping criterion: maximum number of EM Algorithm steps
    skipHess : If True then don't compute BFGS approximation of the inverse Hessian matrix.
        
    OUTPUTS:
    --------
    LATE_em            : Local average treatment effect estimate
    strata_shares_em   : estimates of never-taker, complier and always-taker population shares
    theta_em           : J x 1 vector of model parameter estimates
    iH                 : J x J BFGS approximation of inverse Hessian matrix
    converged          : True if EM algorithm appears to converge to a local max
    
    FUNCTIONS CALLED   : ...display_LATE_mle()...
    ----------------      
    
    """
    
    #-------------------------------------------------------#
    #- First define a series of internal utility functions -#
    #-------------------------------------------------------#
    
    def E_Step(theta, D, Y, Z, X, cLATE, sw):
        
        """
        This function implements the E-Step of the EM-Algorithm. Specifically, given
        the current parameter values it computes the responsibility of each compliance
        strata for each sampled unit; these responsibilities can be used to construct
        the observed (or integrated) log-likelihood.
        """
        
        # basic setup
        K        = X.shape[1]                # number of control variables
        theta    = np.reshape(theta,(-1,1))  # reshape parameter vector into 2d array
        
        # compliance strata (parameter subvectors)
        gamma_c  = theta[0:K,0]              # multinomial logit coefficients for compliers
        gamma_a  = theta[K:2*K,0]            # multinomial logit coefficients for always-takers
        
        # never-taker (parameter subvectors)
        # Y(0)|X,A=n distribution parameters
        beta_n0  = theta[2*K:3*K,0]          
        sigma_n0 = theta[3*K,0]
       
        # complier & always-taker (parameter subvectors)
        if cLATE:
            #--------------------------------------------------------------------------#
            # CASE 1: E[Y(1) - Y(0)|X,A=c] equals a constant. Reduces length of theta
            #         by K-1 
            #--------------------------------------------------------------------------#
            
            # complier (parameter subvector)
            # Y(0)|X,A=c distribution parameters
            beta_c0  = np.append(theta[(3*K+1):(4*K),0], theta[(4*K),0])
            sigma_c0 = theta[(4*K+1),0]
        
            # Y(1)|X,A=c distribution parameters
            # NOTE: slope coefficients in beta_c1 same as those in beta_c0 above (intercept different)
            beta_c1  = np.append(theta[(3*K+1):(4*K),0], theta[(4*K+2),0])
            sigma_c1 = theta[(4*K+3),0]

            # always-taker (parameter subvector)
            # Y(1)|X,A=a distribution parameters
            beta_a1  = theta[(4*K+4):(5*K+4),0]  
            sigma_a1 = theta[(5*K+4),0]
        else:
            #--------------------------------------------------------------------------#
            # CASE 2: Complier Y(0) & Y(1) potential outcome distributions vary flexibly 
            #--------------------------------------------------------------------------#
            
            # complier (parameter subvector)
            # Y(0)|X,A=c distribution parameters
            beta_c0  = theta[(3*K+1):(4*K+1),0]  
            sigma_c0 = theta[(4*K+1),0]
        
            # Y(1)|X,A=c distribution parameters
            beta_c1  = theta[(4*K+2):(5*K+2),0]  
            sigma_c1 = theta[(5*K+2),0]

            # always-taker (parameter subvector)
            # Y(1)|X,A=a distribution parameters
            beta_a1  = theta[(5*K+3):(6*K+3),0]  
            sigma_a1 = theta[(6*K+3),0]
        
        # never-taker (likelihood components)
        p_n      = 1 / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))  
        f_n0     = sp.stats.norm.pdf(Y, X @ beta_n0, sigma_n0)
        
        # complier (likelihood components)
        p_c      = np.exp(X @ gamma_c) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
        f_c0     = sp.stats.norm.pdf(Y, X @ beta_c0, sigma_c0)
        f_c1     = sp.stats.norm.pdf(Y, X @ beta_c1, sigma_c1)
        
        # always-taker (likelihood components)
        p_a      = np.exp(X @ gamma_a) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
        f_a1     = sp.stats.norm.pdf(Y, X @ beta_a1, sigma_a1)
        
        # form strata responsibilities (save them as n x 1 2d numpy arrays, not Pandas Series )
        t_n0     = ((1 - Z) * (1 - D) * (p_n * f_n0) / (p_n * f_n0 + p_c * f_c0) + Z * (1 - D)).values
        t_c0     = ((1 - Z) * (1 - D) * (p_c * f_c0) / (p_n * f_n0 + p_c * f_c0)              ).values
        t_c1     = (Z * D * (p_c * f_c1) / (p_c * f_c1 + p_a * f_a1)                          ).values
        t_a1     = ((1 - Z) * D + Z * D * (p_a * f_a1) / (p_c * f_c1 + p_a * f_a1)            ).values
    
        # compute expected log-likelihood
        E_LogL   = -np.sum(sw * (t_n0 * np.log(f_n0) + t_c0 * np.log(f_c0) + t_c1 * np.log(f_c1) + t_a1 * np.log(f_a1))) \
                   -np.sum(sw * (t_n0 * np.log(p_n)  + (t_c0 + t_c1) * np.log(p_c) + t_a1 * np.log(p_a)))
    
        # Return expected log-likelihood value and responsibility vectors (reshaped into n x 1 2d arrays)
    
        return [E_LogL, t_n0.reshape((-1,1)), t_c0.reshape((-1,1)), t_c1.reshape((-1,1)), t_a1.reshape((-1,1))]
    
    def E_LogL(theta, D, Y, Z, X, cLATE, sw):
        
        """
        This function is used to numerically calculate the observed log-likelihood
        Hessian. It is simply a clean call to E_Step()
        """
        
        elogl, _, _, _, _ = E_Step(theta, D, Y, Z, X, cLATE, sw)
    
        return elogl
    
    def strata_prob_E_LogL(gamma, t, X, sw):
        
        """
        This function forms the portion of Q(theta,theta_current) function 
        associated with the multinomial logit model for the conditional 
        compliance strata probabilities. It is "smoothed" multinomial logit
        log-likelihood (of sorts).
        """
        
        # Get dimension of regressor vector and multinomial logit parameter values
        K       = X.shape[1]                 
        gamma_c = gamma[0:K].reshape((-1,1))
        gamma_a = gamma[K:2*K].reshape((-1,1))
        
        # Get compliance strata responsibility vectors
        t_n, t_c, t_a = t
         
        # Form conditional compliance strata probabilities and log-likelihood    
        p_n      = 1                   / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
        p_c      = np.exp(X @ gamma_c) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
        p_a      = np.exp(X @ gamma_a) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
                
        elogl   = -np.sum(sw * (t_n * np.log(p_n) + t_c * np.log(p_c) + t_a * np.log(p_a)))
                
        return elogl
    
    def strata_prob_E_Score(gamma, t, X, sw):
        
        """
        This function forms the first derivative of Q(theta,theta_current) function 
        with respect to the parameters of the multinomial logit model for the conditional 
        compliance strata probabilities. It is the score vector of a "smoothed" 
        multinomial logit log-likelihood (of sorts).
        """
        
        # Get dimension of regressor vector and multinomial logit parameter values
        K       = X.shape[1]                
        gamma_c = gamma[0:K].reshape((-1,1))
        gamma_a = gamma[K:2*K].reshape((-1,1))
        
        # Get compliance strata responsibility vectors
        t_n, t_c, t_a = t
        
        # Form conditional compliance strata probabilities
        p_n      = 1                   / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
        p_c      = np.exp(X @ gamma_c) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
        p_a      = np.exp(X @ gamma_a) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
        
        # form score vector and save as a 1d array
        escore  = -np.vstack((np.sum(sw * (t_c - p_c) * X, axis = 0).T, np.sum(sw * (t_a - p_a) * X, axis = 0).T))
        escore  = np.ravel(escore, order ='C') 
                
        return escore
            
    def wls(Y, X, t, sw):
        
        """
        Computes the WLS fit of Y onto X given weights (sw * t). This function is used 
        to update the compliance-specific potential outcome distributions during 
        the M-Step of the EM Algorithm.
        """
        
        XX  = (sw * t * X).T @ X
        XY  = (sw * t * X).T @ Y
        beta_hat  = np.linalg.solve(XX, XY) 
        sigma_hat = np.average((Y - X @ beta_hat)**2, weights = np.ravel(sw * t))**(1/2)
       
        return [np.ravel(beta_hat), [sigma_hat]]
    
    #--------------------------------------------------------------------#
    #- MAIN BODY OF FUNCTION                                            -#
    #--------------------------------------------------------------------#
    
    #--------------------------------------------------------------------#
    #- STEP 1 : Organize data                                           -#
    #--------------------------------------------------------------------#
    
    n       = len(Y)                   # Number of observations
    K       = X.shape[1]               # Number of control variables
                    
    # Extract controal variables from pandas data objects
    con_var   = list(X.columns)        # Get control variable names
    
    # Add a constant to the control variable regressor matrix (if needed)
    if not nocons:
        X = X.assign(constant=1)
        con_var.append("constant")
        K      += 1
   
    # Convert X to n x K 2d numpy array
    X = X.values
    
    # Normalize weights to have mean one (if needed)
    if s_wgt is None:
        sw = np.ones((n,1)) 
    else:
        s_wgt_var = s_wgt.name                               # Get sample weight variable name
        sw = np.asarray(s_wgt/s_wgt.mean()).reshape((-1,1))  # Normalized sampling weights with mean one
    
    #--------------------------------------------------------------------#
    #- STEP 2 : Get starting values                                     -#
    #--------------------------------------------------------------------#
    
    # Initialize strata probability "intercept" parameters to match
    # method-of-moments estimates of the fraction of compliers and
    # always-takers in the population. Set all slope parameters to zero.
    
    p_c_sv = D[(Z == 1)].mean() - D[(Z == 0)].mean()
    p_a_sv = D[(Z == 0)].mean()
    
    gamma_c0_sv, gamma_a0_sv = np.log(numpy.linalg.inv([[1-p_c_sv, -p_c_sv],[-p_a_sv, 1-p_a_sv]]) \
                                      @ [[p_c_sv], [p_a_sv]])
    
    gamma_c_sv  = np.append((K-1)*[0], gamma_c0_sv)
    gamma_a_sv  = np.append((K-1)*[0], gamma_a0_sv)
    gamma_sv    = np.concatenate((gamma_c_sv, gamma_a_sv))
    
    # Initialize Y(0) | X, A = n distribution parameters using Z = 1, D = 0 subsample
    beta_n0_sv  = np.append((K-1)*[0], [Y[(Z == 1) & (D == 0)].mean()])
    sigma_n0_sv = [Y[(Z == 1) & (D == 0)].std()]
    
    # Initialize Y(0) | X, A = c distribution parameters using Z = 0, D = 0 subsample
    beta_c0_sv  = np.append((K-1)*[0], [Y[(Z == 0) & (D == 0)].mean()])
    sigma_c0_sv = [Y[(Z == 0) & (D == 0)].std()]
    
    # Initialize Y(1) | X, A = c distribution parameters using Z = 1, D = 1 subsample
    beta_c1_sv  = np.append((K-1)*[0], [Y[(Z == 1) & (D == 1)].mean()])
    sigma_c1_sv = [Y[(Z == 1) & (D == 1)].std()]
    
    # Initialize Y(1) | X, A = a distribution parameters using Z = 0, D = 1 subsample
    beta_a1_sv  = np.append((K-1)*[0], [Y[(Z == 0) & (D == 1)].mean()])
    sigma_a1_sv = [Y[(Z == 0) & (D == 1)].std()]
    
    if cLATE:
        #-----------------------------------------------------------------------------#
        # CASE 1: E[Y(1) - Y(0)|X,A=c] equals a constant. Reduces length of theta_sv
        #         by K-1 
        #-----------------------------------------------------------------------------#
        theta_sv    = np.concatenate([gamma_c_sv, gamma_a_sv, \
                                      beta_n0_sv, sigma_n0_sv, \
                                      (K-1)*[0], [beta_c0_sv[-1]], sigma_c0_sv, [beta_c1_sv[-1]], sigma_c1_sv, \
                                      beta_a1_sv, sigma_a1_sv])     
    
    else:
        #-----------------------------------------------------------------------------#
        # CASE 2: Complier Y(0) & Y(1) potential outcome distributions vary flexibly
        #-----------------------------------------------------------------------------#
        theta_sv    = np.concatenate([gamma_c_sv, gamma_a_sv, \
                                      beta_n0_sv, sigma_n0_sv, beta_c0_sv, sigma_c0_sv, \
                                      beta_c1_sv, sigma_c1_sv, beta_a1_sv, sigma_a1_sv]) 
    
    #--------------------------------------------------------------------#
    #- STEP 3 : EM - Algorithm                                          -#
    #--------------------------------------------------------------------#
    
    finished = False
    converged = False
    logl_sv  = -np.inf
    
    iter = 0  # initialize iteration counter
    
    while not finished:
        
        # ----------------------------------------------------------------------#
        # - E-Step: Compute latent strata responsibilities                     -#
        # ----------------------------------------------------------------------#
        
        # Get current likelihood value and cluster responsibility vectors
        logl, t_n0, t_c0, t_c1, t_a1 = E_Step(theta_sv, D, Y, Z, X, cLATE=cLATE, sw=sw)
        t = [t_n0, t_c0+t_c1, t_a1]
    
        
        # Print optimization output to screen
        if (iter > 0) & (not silent):
            print("Value of expected logL = "       + "%.6f" % -logl + \
                  ",  2-norm of change in theta = " + "%.6f" % delta)
    
        # Calculate likelihood increment and update likelihood value
        Dlogl   = np.abs(logl - logl_sv)
        logl_sv = logl
        
        # ----------------------------------------------------------------------#
        # - M-Step: Update parameter values                                     #
        # ----------------------------------------------------------------------#
        
        # (a) update multinomial logit coefficients for compliance strata membership model
        options_set = {'xtol': xtol, 'maxiter': maxiter, 'disp': False}
        gamma_em_res = sp.optimize.minimize(strata_prob_E_LogL, gamma_sv, args=(t, X, sw), \
                                            method='Newton-CG', jac = strata_prob_E_Score, \
                                            callback = None, options=options_set)
        gamma_em = gamma_em_res.x
        
        
        if cLATE:
            #-----------------------------------------------------------------------------#
            # CASE 1: E[Y(1) - Y(0)|X,A=c] equals a constant. Reduces length of theta_sv
            #         by K-1 
            #-----------------------------------------------------------------------------#
            
            # (b) update strata-specific potential outcome distribution estimates
            # Never-taker and always-taker potential outcome distributions computed as in general
            # case (CASE 2 below)
            beta_n0_em, sigma_n0_em = wls(Y, X, t_n0, sw)
            beta_a1_em, sigma_a1_em = wls(Y, X, t_a1, sw)
            
            # FGLS procedure to compute complier potential outcome distribution parameters
            # under restriction that slope coefficients in Y(1)|X,A=c equal those in Y(0)|X,A=c
            Ym = np.hstack((Y,Y))
            Xm = np.hstack((np.vstack((np.ones((n,1)) , np.zeros((n,1)))), \
                            np.vstack((np.zeros((n,1)), np.ones((n,1)))), \
                            np.vstack((X[:,:-1],X[:,:-1]))))
            
            gls_converged = False
            delta_c_sv =  np.concatenate([(K-1)*[0], [beta_c0_sv[-1]], sigma_c0_sv, [beta_c1_sv[-1]], sigma_c1_sv])
            
            while not gls_converged:
                tm = np.vstack((t_c0/(delta_c_sv[K]**2),t_c1/(delta_c_sv[K+2]**2)))                        # FGLS weights
                swm = np.vstack((sw,sw)) 
                beta_c_em, _ = wls(Ym, Xm, tm, swm)                                                        # Update coefs 
                beta_c0_em   = np.hstack((beta_c_em[2:], beta_c_em[0],))             
                beta_c1_em   = np.hstack((beta_c_em[2:], beta_c_em[1]))
                sigma_c0_em  = [np.average((Y - X @ beta_c0_em)**2, weights = np.ravel(sw * t_c0))**(1/2)] # Update FGLS 
                sigma_c1_em  = [np.average((Y - X @ beta_c1_em)**2, weights = np.ravel(sw * t_c1))**(1/2)] # weights
                delta_c_em    = np.concatenate([beta_c_em[2:], \
                                               [beta_c_em[0]], sigma_c0_em, [beta_c_em[1]], sigma_c1_em])
                # Assess parameter convergence 
                delta_gls = numpy.linalg.norm(delta_c_sv-delta_c_em)
                delta_c_sv = delta_c_em
                
                # Stop when parameters converge
                if (delta_gls < xtol):
                    gls_converged = True
        
            
            # (c) collected updated parameters
            theta_em    = np.concatenate([gamma_em, \
                                          beta_n0_em, sigma_n0_em, \
                                          beta_c_em[2:], [beta_c_em[0]], sigma_c0_em, [beta_c_em[1]], sigma_c1_em, \
                                          beta_a1_em, sigma_a1_em])     
    
        else:
            #-----------------------------------------------------------------------------#
            # CASE 2: Complier Y(0) & Y(1) potential outcome distributions vary flexibly
            #-----------------------------------------------------------------------------#
            
            # (b) update strata-specific potential outcome distribution estimates
            beta_n0_em, sigma_n0_em = wls(Y, X, t_n0, sw)
            beta_c0_em, sigma_c0_em = wls(Y, X, t_c0, sw)
            beta_c1_em, sigma_c1_em = wls(Y, X, t_c1, sw)
            beta_a1_em, sigma_a1_em = wls(Y, X, t_a1, sw)
        
            # (c) collected updated parameters
            theta_em    = np.concatenate([gamma_em, \
                                      beta_n0_em, sigma_n0_em, beta_c0_em, sigma_c0_em, \
                                      beta_c1_em, sigma_c1_em, beta_a1_em, sigma_a1_em]) 
        
        # Assess parameter convergence 
        delta = numpy.linalg.norm(theta_sv-theta_em)
        
        # Check if finished
        if (Dlogl < gtol) | (delta < xtol) | (iter >= maxiter):
            finished = True
            if (Dlogl < gtol) | (delta < xtol):
                converged = True
        
        # Update parameters
        theta_sv = theta_em
        iter += 1
    
    # compute MLEs of compliance strata shares
    strata_shares_em = [np.mean(sw * t_n0), np.mean(sw * (t_c0+t_c1)), np.mean(sw * t_a1)]
    
    #--------------------------------------------------------------------#
    #- STEP 4 : Compute variance-covariance matrix                      -#
    #--------------------------------------------------------------------#
    
    if not skipHess:
        if not silent:
            print("")
            print("--------------------------------------------------------------")
            print("- Computing approximate inverse Hessian using BFGS algorithm -")
            print("--------------------------------------------------------------")
        # Using numerical estimate of log-likelihood Hessian for standard errors
        # Use the BGFS algorithm to get approximation (which will be positive definite by construction)
        options_set = {'gtol': gtol, 'maxiter': maxiter, 'disp': silent, 'norm': 10*delta}
        opt_res = sp.optimize.minimize(E_LogL, theta_em, args=(D, Y, Z, X, cLATE, sw), \
                                       method='bfgs', jac = None, \
                                       callback = None, options=options_set)
        if not opt_res.success:
            print("")
            print("WARNING: BFGS approximation to inverse Hessian may be inaccurate.")
        iH = opt_res.hess_inv
    else:
        iH = np.NaN*np.ones((len(theta_em),len(theta_em)))
    
    #--------------------------------------------------------------------#
    #- STEP 5 : Compute LATE                                            -#
    #--------------------------------------------------------------------#
    
    if cLATE:
        beta_c0_em = np.concatenate([theta_em[(3*K+1):(4*K)], [theta_em[4*K]]])
        beta_c1_em = np.concatenate([theta_em[(3*K+1):(4*K)], [theta_em[4*K+2]]])
        LATE_em = [np.average(X @ beta_c1_em - X @ beta_c0_em, weights = np.ravel(sw * (t_c0+t_c1)))]
        var_LATE_em = (iH[4*K,4*K] + iH[4*K+1,4*K+1] - 2*iH[4*K,4*K+1])
    else:
        beta_c0_em = theta_em[(3*K+1):(4*K+1)]  
        beta_c1_em = theta_em[(4*K+2):(5*K+2)]
        LATE_em = [np.average(X @ beta_c1_em - X @ beta_c0_em, weights = np.ravel(sw * (t_c0+t_c1)))]
        var_LATE_em = np.nan
    
    #--------------------------------------------------------------------#
    #- STEP 6 : Display results if requested                            -#
    #--------------------------------------------------------------------#
    
    if not silent:
        display_LATE_mle(theta_em, iH, LATE_em, var_LATE_em, strata_shares_em, np.NaN*np.ones((3,3)), \
                         D, Y, Z, pd.DataFrame(X,columns=con_var), cLATE=cLATE, nocons=True)
        print("")  
        print("Log-Likelihood,          logl : " + "{:<15,.1f}".format(-logl))
        
        if s_wgt is not None:
            print("NOTE: (Sampling) weighted MLEs computed.")
            print("      Weight-variable   = " + s_wgt_var)    
 
   
    return [LATE_em, strata_shares_em, theta_em, iH, converged]

In [11]:
def display_LATE_mle(theta, vcov_theta, LATE, var_LATE, strata_shares, vcov_strata_shares, \
                     D, Y, Z, X, cLATE=False, nocons=True):
    
    """
    This function display estimation results produced by the LATE_mle() function.
    See that functions documentation for details on parameters. Some of the above
    parameters are generally computed as a by product of the parametric bootstrap
    as implemented by the bsLATE_mle() function.
    """
    
    n       = len(Y)                   # Number of observations
    K       = X.shape[1]               # Number of control variables
                    
    # Extract variable names from pandas data objects
    out_var   = Y.name                 # Get outcome variable names
    treat_var = D.name                 # Get treatment variable name
    inst_var  = Z.name                 # Get instrumental variable name
    con_var   = list(X.columns)        # Get control variable names
    
    # Add a constant to the control variable regressor matrix (if needed)
    if not nocons:
        X = X.assign(constant=1)
        K += 1
    
    if cLATE:
        #-----------------------------------------------------------------------------#
        # CASE 1: E[Y(1) - Y(0)|X,A=c] equals a constant. Reduces length of theta_sv
        #         by K-1
        #-----------------------------------------------------------------------------#
        
        # compliance strata parameters
        gamma_c  = theta[0:K]              # multinomial logit coefficients for compliers
        stderr_gamma_c = np.diag(vcov_theta[0:K,0:K])**(1/2)
    
        gamma_a  = theta[K:2*K]            # multinomial logit coefficients for always-takers
        stderr_gamma_a = np.diag(vcov_theta[K:2*K,K:2*K])**(1/2)
    
        # never-takers potential outcome parameters
        # Y(0)|X,A=n distribution parameters
        delta_n0 = theta[2*K:(3*K+1)]      
        stderr_delta_n0 = np.diag(vcov_theta[2*K:(3*K+1),2*K:(3*K+1)])**(1/2)
    
        # compliers potential outcome parameters
        # Y(0)|X,A=c distribution parameters
        delta_c0 = np.concatenate([theta[(3*K+1):(4*K)], [theta[4*K]], [theta[4*K+1]]])
        stderr_delta_c0 = np.concatenate([np.diag(vcov_theta[(3*K+1):(4*K),(3*K+1):(4*K)])**(1/2), \
                                         [vcov_theta[4*K,4*K]**(1/2)], [vcov_theta[4*K+1,4*K+1]**(1/2)]])
        
        # Y(1)|X,A=c distribution parameters
        delta_c1 = np.concatenate([theta[(3*K+1):(4*K)], [theta[4*K+2]], [theta[4*K+3]]])
        stderr_delta_c1 = np.concatenate([np.diag(vcov_theta[(3*K+1):(4*K),(3*K+1):(4*K)])**(1/2), \
                                         [vcov_theta[4*K+2,4*K+2]**(1/2)], [vcov_theta[4*K+3,4*K+3]**(1/2)]])
    
        # always-takers potential outcome parameters
        delta_a1  = theta[(4*K+4):(5*K+5)] # Y(1)|X,A=a distribution parameters
        stderr_delta_a1 = np.diag(vcov_theta[(4*K+4):(5*K+5),(4*K+4):(5*K+5)])**(1/2)
        
    else:
        #-----------------------------------------------------------------------------#
        # CASE 2: Complier Y(0) & Y(1) potential outcome distributions vary flexibly
        #-----------------------------------------------------------------------------#
        
        # compliance strata parameters
        gamma_c  = theta[0:K]              # multinomial logit coefficients for compliers
        stderr_gamma_c = np.diag(vcov_theta[0:K,0:K])**(1/2)
    
        gamma_a  = theta[K:2*K]            # multinomial logit coefficients for always-takers
        stderr_gamma_a = np.diag(vcov_theta[K:2*K,K:2*K])**(1/2)
    
        # never-takers potential outcome parameters
        # Y(0)|X,A=n distribution parameters
        delta_n0 = theta[2*K:(3*K+1)]      
        stderr_delta_n0 = np.diag(vcov_theta[2*K:(3*K+1),2*K:(3*K+1)])**(1/2)
    
        # compliers potential outcome parameters
        # Y(0)|X,A=c distribution parameters
        delta_c0 = theta[(3*K+1):(4*K+2)]  
        stderr_delta_c0 = np.diag(vcov_theta[(3*K+1):(4*K+2),(3*K+1):(4*K+2)])**(1/2)
    
        # Y(1)|X,A=c distribution parameters
        delta_c1 = theta[(4*K+2):(5*K+3)]  
        stderr_delta_c1 = np.diag(vcov_theta[(4*K+2):(5*K+3),(4*K+2):(5*K+3)])**(1/2)
    
        # always-takers potential outcome parameters
        # Y(1)|X,A=a distribution parameters
        delta_a1  = theta[(5*K+3):(6*K+4)] 
        stderr_delta_a1 = np.diag(vcov_theta[(5*K+3):(6*K+4),(5*K+3):(6*K+4)])**(1/2)

    # Pandas dataframe of multinomial logit estimates of compliance strata model
    strata_results = pd.DataFrame(np.vstack((gamma_c, stderr_gamma_c, gamma_a, stderr_gamma_a)).T, con_var, \
                                  ['compliers', 's.e.', 'always-takers', 's.e.'])
    
    # Pandas dataframe of compliance strata shares and standard errors
    strata_share_results=pd.DataFrame({'share': strata_shares, 's.e.': np.diag(vcov_strata_shares)},\
                                      index=['never-takers','compliers','always-takers'])
    
    # Compute and collect potential outcome distribution parameter estimates into a Pandas dataframe
    con_var.append("sigma")
    potential_outcome_results = pd.DataFrame(np.vstack((delta_n0, stderr_delta_n0, delta_c0, stderr_delta_c0, \
                                                        delta_c1, stderr_delta_c1, delta_a1, stderr_delta_a1)).T, \
                                             con_var, \
                                             ['Y(0): never-takers', 's.e.', 'Y(0): compliers', 's.e.', \
                                              'Y(1): compliers',    's.e.', 'Y(1): always-takers', 's.e.'])
    
    
    print("----------------------------------------------------------------------------")
    print("- Model-based IV estimates                                                 -")
    print("----------------------------------------------------------------------------")
    print("Number of observations      n : " + "{:<15,.0f}".format(n))
    print("Outcome,                    Y : " + out_var)
    print("Treatment (binary),         D : " + treat_var)
    print("Instrument (binary),        Z : " + inst_var)
    print("(see results below for list of pre-treament control variables)")
    print("----------------------------------------------------------------------------")
    print("")
    print("----------------------------------------------------------------------------")
    print("- Multinomial logit model for conditional compliance strata membership     -")
    print("----------------------------------------------------------------------------")
    print("")
    print(strata_results)
    print("")
    print("Estimated compliance strata population shares:")
    print("----------------------------------------------")
    print("")
    print(strata_share_results)
    print("")
    print("----------------------------------------------------------------------------")
    print("- Gaussian models for potential outcome distributions                      -")
    print("----------------------------------------------------------------------------")
    print("")
    print(potential_outcome_results)
    print("")
    print("----------------------------------------------------------------------------")
    print("- Model-based Local Average Treatement Effect Estimate                     -")
    print("----------------------------------------------------------------------------")
    print("LATE  :  " + "{:>10,.4f}".format(LATE[0]))
    print("s.e.  : (" + "{:>10,.4f}".format(var_LATE**(1/2)) + ")")
    print("----------------------------------------------------------------------------")
    
    return None

In [12]:
def bsLATE_mle(theta_em, X, Z, cLATE=False, nocons=False, s_wgt=None, silent=False, B=100, gtol=1e-2, xtol=1e-4, maxiter=1000):
    
    """
    This function implements a parametric boostrap procedure for the LATE_mle
    command. See that function's documentation for more details on the
    above parameters. The parametric bootstrap is the recommended method
    for construction standard errors and confidence intervals.
    """
    
    #--------------------------------------------------------------------#
    #- STEP 1 : Organize data & model parameters for simulation         -#
    #--------------------------------------------------------------------#
    
    [n, K]    = X.shape                # Number of observations & control variables
    con_var   = list(X.columns)        # Get control variable names
    
    # Add a constant to the control variable regressor matrix (if needed)
    if not nocons:
        X = X.assign(constant=1)
        con_var.append("constant")
        K      += 1
    
    theta    = np.reshape(theta_em,(-1,1)) # reshape parameter into 2d array
        
    # compliance strata (parameter subvectors)
    gamma_c  = theta[0:K,0]                # multinomial logit coefficients for compliers
    gamma_a  = theta[K:2*K,0]              # multinomial logit coefficients for always-takers
        
    # never-taker (parameter subvectors)
    # Y(0)|X,A=n distribution parameters
    beta_n0  = theta[2*K:3*K,0]          
    sigma_n0 = theta[3*K,0]
       
    # complier & always-taker (parameter subvectors)
    if cLATE:
        #--------------------------------------------------------------------------#
        # CASE 1: E[Y(1) - Y(0)|X,A=c] equals a constant. Reduces length of theta
        #         by K-1 
        #--------------------------------------------------------------------------#
            
        # complier (parameter subvector)
        # Y(0)|X,A=c distribution parameters
        beta_c0  = np.append(theta[(3*K+1):(4*K),0], theta[(4*K),0])
        sigma_c0 = theta[(4*K+1),0]
        
        # Y(1)|X,A=c distribution parameters
        # NOTE: slope coefficients in beta_c1 same as those in beta_c0 above
        beta_c1  = np.append(theta[(3*K+1):(4*K),0], theta[(4*K+2),0])
        sigma_c1 = theta[(4*K+3),0]

        # always-taker (parameter subvector)
        # Y(1)|X,A=a distribution parameters
        beta_a1  = theta[(4*K+4):(5*K+4),0]  
        sigma_a1 = theta[(5*K+4),0]
    else:
        #--------------------------------------------------------------------------#
        # CASE 2: Complier Y(0) & Y(1) potential outcome distributions vary flexibly 
        #--------------------------------------------------------------------------#
            
        # complier (parameter subvector)
        # Y(0)|X,A=c distribution parameters
        beta_c0  = theta[(3*K+1):(4*K+1),0]  
        sigma_c0 = theta[(4*K+1),0]
        
        # Y(1)|X,A=c distribution parameters
        beta_c1  = theta[(4*K+2):(5*K+2),0]  
        sigma_c1 = theta[(5*K+2),0]

        # always-taker (parameter subvector)
        # Y(1)|X,A=a distribution parameters
        beta_a1  = theta[(5*K+3):(6*K+3),0]  
        sigma_a1 = theta[(6*K+3),0]
    
    # Compute compliance strata conditional probabilities
    p_n      = 1 / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
    p_c      = np.exp(X @ gamma_c) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
    p_a      = np.exp(X @ gamma_a) / (1+np.exp(X @ gamma_c) + np.exp(X @ gamma_a))
    
    # initialize matrix to store bootstrap results (LATE, compliance strata shares and theta)
    Bootstrap_Results = [np.zeros((B,1)), np.zeros((B,3)), np.zeros((B,len(theta_em)))]
    
    for b in range(0,B):
        start = time.time()
        
        # Simulate compliance types
        U   = np.random.uniform(0,1,(n,))
        A_n = 1*(U<=p_n)
        A_c = 1*(U>p_n)*(U<=(p_n+p_c))
        A_a = 1*(U>(p_n+p_c))
        
        # Reconstruct/simulate treatment choices based on simulated types
        # NOTE: choice is deterministic given compliance type and instrument value
        D = 0*A_n + Z*A_c + 1*A_a
        
        # Simulate outcomes based on types and treatments
        Y0n = np.random.normal(X @ beta_n0, sigma_n0)
        Y0c = np.random.normal(X @ beta_c0, sigma_c0)
        Y1c = np.random.normal(X @ beta_c1, sigma_c1)
        Y1a = np.random.normal(X @ beta_a1, sigma_a1)
        Y   = Y0n*A_n + (1-D)*A_c*Y0c + D*A_c*Y1c + Y1a*A_a
        
        D = pd.Series(D, name="D")
        Y = pd.Series(Y, name="Y")
        
        # Fit model to simulated data
        [LATE_b, strata_shares_b, theta_b, _, _] = LATE_mle(D, Y, X, Z, cLATE=cLATE, nocons=True, s_wgt=s_wgt, \
                                                            silent=True, \
                                                            theta_sv=theta_em, gtol=gtol, xtol=xtol, \
                                                            maxiter=maxiter, skipHess=True)
        
        # Store results from bth bootstrap replication
        Bootstrap_Results[0][b,:] = LATE_b
        Bootstrap_Results[1][b,:] = strata_shares_b
        Bootstrap_Results[2][b,:] = theta_b
        
        end = time.time()
        if (b+1) % 1 == 0:
            print("Time required f/ boostrap rep  " + \
                  str(b+1) + " of " + str(B) + ": " + str(end-start))   
        
        var_LATE = np.cov(Bootstrap_Results[0], rowvar=False)
        vcov_strata_shares = np.cov(Bootstrap_Results[1], rowvar=False)
        vcov_theta = np.cov(Bootstrap_Results[2], rowvar=False)
        
        
    return [Bootstrap_Results, var_LATE, vcov_strata_shares, vcov_theta]

In [13]:
# Hide some categories of warning messages
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

I now fit the same model (albeit with additional parametric assumptions, but also more flexible covariate interactions) by maximum likelihood. The LATE point estimate is similar to the method of moments one. This is striking as the LATE is allowed to vary with pre-treatment covariates in this model. An attempt to compute standard errors using the BFGS approximation to the inverse Hessian was not successful. Below I implement a parametric bootstrap procedure in order to compute standard errors.

In [14]:
LATE_em, strata_shares_em, theta_em, iH, converged = LATE_mle(D, Y, X, Z, cLATE=False, nocons=True, s_wgt=sw)

Value of expected logL = -2924641.414423,  2-norm of change in theta = 3.630936
Value of expected logL = -2909333.396233,  2-norm of change in theta = 0.900183
Value of expected logL = -2898662.273690,  2-norm of change in theta = 0.352486
Value of expected logL = -2889710.961960,  2-norm of change in theta = 0.415899
Value of expected logL = -2886047.856142,  2-norm of change in theta = 1.138297
Value of expected logL = -2882690.918767,  2-norm of change in theta = 0.097940
Value of expected logL = -2881080.379958,  2-norm of change in theta = 0.083616
Value of expected logL = -2877457.694596,  2-norm of change in theta = 1.010942
Value of expected logL = -2877647.387872,  2-norm of change in theta = 0.164236
Value of expected logL = -2877848.805856,  2-norm of change in theta = 0.113498
Value of expected logL = -2877744.514170,  2-norm of change in theta = 0.075852
Value of expected logL = -2877108.507078,  2-norm of change in theta = 0.065785
Value of expected logL = -2875593.380224

Next I use the parametric bootstrap to estimate standard errors for the LATE maximum likelihood estimates. I use 100 bootstrap replications. Since the EM Algorithm can be slow to converge, the bootstrap can be computationally intensive even for modest size datasets.

In [34]:
[Bootstrap_Results, var_LATE, vcov_strata_shares, vcov_theta] = bsLATE_mle(theta_em, X, Z, cLATE=False, \
                                                                           nocons=True, s_wgt=sw, silent=False, B=100, \
                                                                           gtol=1e-2, xtol=1e-4, maxiter=1000)

Time required f/ boostrap rep  1 of 100: 3.8735570907592773
Time required f/ boostrap rep  2 of 100: 1.6632046699523926
Time required f/ boostrap rep  3 of 100: 4.126051902770996
Time required f/ boostrap rep  4 of 100: 4.6180901527404785
Time required f/ boostrap rep  5 of 100: 4.377985000610352
Time required f/ boostrap rep  6 of 100: 3.4339520931243896
Time required f/ boostrap rep  7 of 100: 4.236380100250244
Time required f/ boostrap rep  8 of 100: 3.736760139465332
Time required f/ boostrap rep  9 of 100: 2.176785945892334
Time required f/ boostrap rep  10 of 100: 0.592965841293335
Time required f/ boostrap rep  11 of 100: 39.25439476966858
Time required f/ boostrap rep  12 of 100: 2.6712071895599365
Time required f/ boostrap rep  13 of 100: 2.902730941772461
Time required f/ boostrap rep  14 of 100: 32.32994198799133
Time required f/ boostrap rep  15 of 100: 3.484834909439087
Time required f/ boostrap rep  16 of 100: 46.53768491744995
Time required f/ boostrap rep  17 of 100: 2.

The bootstrap standard errors suggest that the model-based LATE estimate is also rather imprecisely estimated. In contrast the compliance strata shares are precisely determined.

In [35]:
display_LATE_mle(theta_em, vcov_theta, LATE_em, var_LATE, strata_shares_em, vcov_strata_shares, \
                 D, Y, Z, X, cLATE=False, nocons=True)

----------------------------------------------------------------------------
- Model-based IV estimates                                                 -
----------------------------------------------------------------------------
Number of observations      n : 1,480          
Outcome,                    Y : lwage76
Treatment (binary),         D : ed76
Instrument (binary),        Z : nearc4
(see results below for list of pre-treament control variables)
----------------------------------------------------------------------------

----------------------------------------------------------------------------
- Multinomial logit model for conditional compliance strata membership     -
----------------------------------------------------------------------------

          compliers    s.e.  always-takers    s.e.
age76        0.0098  0.0124         0.0090  0.0084
smsa66r      0.1248  0.2592        -0.3526  0.1866
constant    -1.7815  0.4011        -0.8213  0.2222

Estimated compliance strata